In [1]:
# Run if using on google collab, change path if using on a copy
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
path = '/content/drive/MyDrive/[02] School/[01] University/[02] Bachelor\'s Year 2/[02] Spring Semester/[04] Machine Learning/Colab Notebooks/ML - LGI/Project/'

Mounted at /content/drive


In [3]:
# run if running locally
path = ''

In [4]:
#standard initial libraries
from math import ceil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import chi2_contingency

#sk-learn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier;


#setting seaborn visual style in plt
sns.set_theme()

# Integration & Exploration

In [5]:
#import the data
traindata: pd.DataFrame = pd.read_csv(path + 'project_data/train.csv').set_index('Userid').drop('Observations', axis=1).drop_duplicates()
testdata: pd.DataFrame = pd.read_csv(path + 'project_data/test.csv').set_index('Userid').drop(['Registered', 'Observations'], axis=1)

traindata = traindata[traindata['Registered'] == 'Yes']
traindata = traindata.drop('Registered', axis=1)

In [6]:
metricFeatures: list[str] = ['Application order','Previous qualification score','Entry score','Age at enrollment','N units credited 1st period',
       'N units taken 1st period', 'N scored units 1st period',
       'N units approved 1st period', 'Average grade 1st period',
       'N unscored units 1st period', 'N units credited 2nd period',
       'N units taken 2nd period', 'N scored units 2nd period',
       'N units approved 2nd period', 'Average grade 2nd period',
       'N unscored units 2nd period','Social Popularity']
categoricalFeatures: list[str] =  ['Application mode','Marital status','Course','Previous qualification','Nationality',"Mother's qualification",
       "Father's qualification","Mother's occupation","Father's occupation"]
boolFeatures: list[str] = ['Morning shift participation','Displaced','Special needs','Debtor','Regularized Fees','Gender_Male','External Funding','International']

In [ ]:
traindata.info()

In [ ]:
for variable in categoricalFeatures:
  print(traindata[variable].value_counts())

In [50]:
X = traindata.drop(['Success'], axis = 1)
y = traindata['Success']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 15, test_size = 0.2)

y_train.replace({'Succeeded': 2, 'Holding on': 1, 'Gave up': 0}, inplace = True)
X_train_filled = X_train.fillna(X_train[metric_features].median())
X_train_filled = X_train_filled.fillna(X_train_filled[categorical_features].mode().iloc[0])
X_train_filled.drop(remlist, axis = 1, inplace = True)
X_train_filled = pd.get_dummies(X_train_filled, drop_first = True)
# X_train_filled.drop(['N units taken 2nd period', 'N units credited 2nd period', 'N units approved 2nd period'], axis = 1, inplace = True)
scaler = MinMaxScaler().fit(X_train_filled)
X_train_scaled = scaler.transform(X_train_filled)
X_train = pd.DataFrame(X_train_scaled, columns = X_train_filled.columns, index = X_train_filled.index)

y_val.replace({'Succeeded': 2, 'Holding on': 1, 'Gave up': 0}, inplace = True)
X_val_filled = X_val.fillna(X_val[metric_features].median())
X_val_filled = X_val_filled.fillna(X_val_filled[categorical_features].mode().iloc[0])
X_val_filled.drop(remlist, axis = 1, inplace = True)
X_val_filled = pd.get_dummies(X_val_filled, drop_first = True)
# X_val_filled.drop(['N units taken 2nd period', 'N units credited 2nd period', 'N units approved 2nd period'], axis = 1, inplace = True)
X_val_scaled = scaler.transform(X_val_filled)
X_val = pd.DataFrame(X_val_scaled, columns = X_val_filled.columns, index = X_val_filled.index)

testdata_filled = testdata.fillna(testdata[metric_features].median())
testdata_filled = testdata_filled.fillna(testdata_filled[categorical_features].mode().iloc[0])
# testdata_filled.drop('Observations', axis = 1, inplace = True)
testdata_filled.drop(remlist, axis = 1, inplace = True)
testdata_filled = pd.get_dummies(testdata_filled, drop_first = True)
# testdata_filled.drop(['N units taken 2nd period', 'N units credited 2nd period', 'N units approved 2nd period'], axis = 1, inplace = True)
testdata_scaled = scaler.transform(testdata_filled)
testdata = pd.DataFrame(testdata_scaled, columns = testdata_filled.columns, index = testdata_filled.index)

In [19]:
y.replace({'Succeeded': 2, 'Holding on': 1, 'Gave up': 0}, inplace = True)
# X_filled = X.fillna(X[metric_features].median())
# X_filled = X_filled.fillna(X_filled[categorical_features].mode().iloc[0])
# X_filled.drop(remlist, axis = 1, inplace = True)
X_filled = pd.get_dummies(X, dummy_na = False, drop_first = False)
X_train = X_filled
# X_train_filled.drop(['N units taken 2nd period', 'N units credited 2nd period', 'N units approved 2nd period'], axis = 1, inplace = True)

In [20]:
# testdata_filled = testdata.fillna(testdata[metric_features].median())
# testdata_filled = testdata_filled.fillna(testdata_filled[categorical_features].mode().iloc[0])
# testdata_filled.drop(remlist, axis = 1, inplace = True)
testdata_filled = pd.get_dummies(testdata, dummy_na=False, drop_first = False)
testdata = testdata_filled
# testdata_filled.drop(['N units taken 2nd period', 'N units credited 2nd period', 'N units approved 2nd period'], axis = 1, inplace = True)

In [ ]:
columns_validation = set(X_train.columns) - set(X_val.columns)

for col in columns_validation:
    X_val[col] = 0

for column in X_val.columns:
    if column not in X_train.columns:
        X_val.drop(column, axis = 1, inplace = True)

X_val = X_val[X_train.columns]

In [21]:
columns_validation = set(X_train.columns) - set(testdata.columns)

for col in columns_validation:
  testdata[col] = 0

for column in testdata.columns:
  if column not in X_train.columns:
    testdata.drop(column, axis = 1, inplace = True)

testdata = testdata[X_train.columns]

In [ ]:
X_train.isna().sum()

In [ ]:
for i, col in enumerate(metric_features):
  plt.figure(i)
  sns.boxplot(x=col, data=X_train)

# Preprocessing

## Get Dummy variables

## Missing Values

In [ ]:
X_train_filled = X_train.fillna(X_train[metric_features].median())

In [ ]:
X_train_filled = X_train_filled.fillna(X_train_filled[categorical_features].mode().iloc[0])

In [ ]:
for i, col in enumerate(metric_features):
  plt.figure(i)
  sns.boxplot(x=col, data=X_train_filled)

In [ ]:
X_train_filled.isna().sum()

# Feature Selection

In [ ]:
univariatecheck = X_train[metric_features].var()

In [ ]:
for i in univariatecheck:
  print(np.round(i,5))

In [ ]:
def cor_heatmap(cor):
    plt.figure(figsize=(12,10))
    sns.heatmap(data = cor, annot = True, cmap = plt.cm.Reds, fmt='.1')
    plt.show()

In [ ]:
cor_spearman = pd.concat([X_train[metric_features], y_train], axis=1).corr(method ='spearman')
cor_heatmap(cor_spearman)

In [ ]:
cor_kendall = pd.concat([X_train[metric_features], y_train], axis=1).corr(method ='kendall')
cor_heatmap(cor_kendall)

In [ ]:
anova = SelectKBest(f_classif, k=5)

In [ ]:
X_anova = anova.fit_transform(X_train_filled[metric_features], y_train)

In [ ]:
selected_features = pd.Series(anova.get_support(), index = X_train_filled[metric_features].columns)
selected_features

In [ ]:
cat_spearman = pd.concat([X_train[[s for s in X_train.columns if s.startswith("Mother's qualification")]], y_train], axis = 1).corr(method = 'spearman')
cor_heatmap(cat_spearman)

In [ ]:
cat_spearman = pd.concat([X_train[[s for s in X_train.columns if s.startswith("Father's qualification")]], y_train], axis = 1).corr(method = 'spearman')
cor_heatmap(cat_spearman)

In [ ]:
cat_spearman = pd.concat([X_train[[s for s in X_train.columns if s.startswith("Marital status")]], y_train], axis = 1).corr(method = 'spearman')
cor_heatmap(cat_spearman)

In [ ]:
cat_spearman = pd.concat([X_train[[s for s in X_train.columns if s.startswith("Course")]], y_train], axis = 1).corr(method = 'spearman')
cor_heatmap(cat_spearman)

In [ ]:
cat_spearman = pd.concat([X_train[[s for s in X_train.columns if s.startswith("Previous qualification")]], y_train], axis = 1).corr(method = 'spearman')
cor_heatmap(cat_spearman)

In [ ]:
cat_spearman = pd.concat([X_train[[s for s in X_train.columns if s.startswith("Mother's occupation")]], y_train], axis = 1).corr(method = 'spearman')
cor_heatmap(cat_spearman)

In [ ]:
cat_spearman = pd.concat([X_train[[s for s in X_train.columns if s.startswith("Father's occupation")]], y_train], axis = 1).corr(method = 'spearman')
cor_heatmap(cat_spearman)

In [ ]:
cat_spearman = pd.concat([X_train[[s for s in X_train.columns if s.startswith("Nationality")]], y_train], axis = 1).corr(method = 'spearman')
cor_heatmap(cat_spearman)

In [ ]:
cat_spearman = pd.concat([X_train[["External Funding", "Application mode", "Morning shift participation", "Displaced", "Special needs", "Debtor", "Regularized Fees", "Gender_Male", "International"]], y_train], axis = 1).corr(method = 'spearman')
cor_heatmap(cat_spearman)

In [ ]:
ls = X_train.columns
for i in ls:
    print(i)

| Predictor                    | Variance | Spearman | Kendall  | ANOVA   | Solution |
|------------------------------|----------|----------|----------|---------|----------|
| Application order            | Keep     | Keep     | Keep     | Discard | Keep     |
| Previous qualification score | Keep     | Keep     | Keep     | Discard | Keep     |
| Entry score                  | Keep     | Keep     | Keep     | Discard | Keep     |
| Age at enrollment            | Keep     | Keep     | Keep     | Keep    | Keep     |
| N units credited 1st period  | Keep     | Keep     | Keep     | Discard | Keep     |
| N units taken 1st period     | Keep     | Keep     | Keep     | Discard | Keep     |
| N scored units 1st period    | Keep     | Keep     | Keep     | Discard | Keep     |
| N units approved 1st period  | Keep     | Keep     | Keep     | Keep    | Keep     |
| Average grade 1st period     | Keep     | Keep     | Keep     | Keep    | Keep     |
| N unscored units 1st period  | Keep     | Keep     | Keep     | Discard | Keep     |
| N units credited 2nd period  | Keep     | Discard  | Discard  | Discard | Discard  |
| N units taken 2nd period     | Keep     | Discard  | Discard  | Discard | Discard  |
| N scored units 2nd period    | Keep     | Keep     | Keep     | Discard | Keep     |
| N units approved 2nd period  | Keep     | Discard  | Discard? | Keep    | Test     |
| Average grade 2nd period     | Keep     | Discard? | Keep     | Keep    | Test     |
| N unscored units 2nd period  | Keep     | Keep     | Keep     | Discard | Keep     |
| Social Popularity            | Keep     | Keep     | Keep     | Discard | Keep     |

# Model

In [ ]:
parameter_space_lr = {
    'penalty': ["l2", "elasticnet", None],
    'solver': ["lbfgs", "liblinear", "newton-cg", "newton-cholesky", "sag", "saga"],
    'class_weight': [None, 'balanced'],
    'C' : [0.2, 0.5, 0.7, 1], # inverse regularization parameter
}

In [ ]:
#running this will give some warnings when convergence is not reached- run the code below to have them not show up
import warnings
warnings.filterwarnings('ignore')

# clf = GridSearchCV(lr, parameter_space_lr, scoring = 'f1', cv = 10,)
# clf.fit(X_train,y_train)

# Best parameter set
print('------------------------------------------------------------------------------------------------------------------------')
# print('Best parameters found:\n', clf.best_params_)
print('Best parameters found:\n', gsdtmicro.best_params_)
print('Best parameters found:\n', gsdtmacro.best_params_)
print('Best parameters found:\n', gsdtweighted.best_params_)
# print('Best parameters found:\n', gsmlpc.best_params_)
print('------------------------------------------------------------------------------------------------------------------------')


------------------------------------------------------------------------------------------------------------------------
Best parameters found:
 {'criterion': 'entropy', 'max_depth': 4, 'splitter': 'best'}
Best parameters found:
 {'criterion': 'gini', 'max_depth': 4, 'splitter': 'best'}
Best parameters found:
 {'criterion': 'gini', 'max_depth': 10, 'splitter': 'random'}
------------------------------------------------------------------------------------------------------------------------


In [22]:
model = RandomForestClassifier(800)
model.fit(X_train, y)

RandomForestClassifier(n_estimators=800)

In [23]:
test_pred = model.predict(testdata) #test preds

In [ ]:
index = testdata.index.T
answer = pd.DataFrame([index, test_pred]).T
answer.columns = ['Userid','Success']
answer.replace({'Success': {0: 'Gave Up', 1: 'Holding on', 2: 'Succeeded'}}, inplace = True)
answer.set_index('Userid').to_csv('answer.csv')

---

In [111]:
#import the data and drop remove useless stuff
traindata: pd.DataFrame = pd.read_csv(path + 'project_data/train.csv').set_index('Userid').drop('Observations', axis=1).drop_duplicates()
testdata: pd.DataFrame = pd.read_csv(path + 'project_data/test.csv').set_index('Userid').drop(['Registered', 'Observations'], axis=1)

traindata = traindata[traindata['Registered'] == 'Yes']
traindata = traindata.drop('Registered', axis=1)

In [112]:
X: pd.DataFrame = traindata.drop(['Success'], axis = 1)
y: pd.Series = traindata['Success']

In [113]:
def fillNa(data: pd.DataFrame) -> pd.DataFrame:
    """Fill missing values

    Args:
        data (`pd.DataFrame`): Dataframe to be treated

    Returns:
        `pd.DataFrame`: Treated dataframe
    """    

    # on all of these features, if a value were to be different than 0, then it would not be missing
    ifNaThen0: tuple[str,...] = (
        "N units credited 1st period",
        "N unscored units 1st period",
        "N units approved 1st period",
        "N units credited 2nd period",
        "N unscored units 2nd period",
        "N units approved 2nd period"
    )

    # these features are filled differently, basically incoherence checking, but filling the Na on 'N units approved 1st/2nd period' is needed beforehand, more info below
    checkAfterVars: list[list[str,str]] = [
        ["N units taken 1st period", "N units approved 1st period"],
        ["N units taken 2nd period", "N units approved 2nd period"]
    ] 

    for var in metricFeatures:
        if var in checkAfterVars: continue # skip current iteration

        if var in ifNaThen0:
            data[var] = data[var].fillna(0) # fill the ifNaThen0 vars with well, 0s
        else:    
            data[var] = data[var].fillna(data[var].median()) # fill everything else with the median of the values of the feature

    for varlist in checkAfterVars:
        # search for Na values on N units taken and replace by the equivalent value on N units approved
        data.loc[data[varlist[0]].isna(), varlist[0]] = data[varlist[1]]
        # search for values on N units taken that are smaller than the equivalent on N units approved, replace by the equivalent value on N units approved
        data.loc[data[varlist[0]] < data[varlist[1]] , varlist[0]] = data[varlist[1]]

    return data

In [114]:
fillNa(X)
fillNa(testdata)

,Marital status,Application mode,Application order,Course,Morning shift participation,Previous qualification,Previous qualification score,Nationality,Mother's qualification,Father's qualification,...,N units approved 1st period,Average grade 1st period,N unscored units 1st period,N units credited 2nd period,N units taken 2nd period,N scored units 2nd period,N units approved 2nd period,Average grade 2nd period,N unscored units 2nd period,Social Popularity
Userid,,,,,,,,,,,,,,,,,,,,,
2365,single,1.0,1.0,Agronomy,1.0,12th grade,144.0,Portuguese,9th grade,9th grade,...,4.0,12.000000,2.0,0.0,6.0,10.0,4.0,12.500000,0.0,42
2554,single,2.0,1.0,Management (evening attendance),0.0,12th grade,160.0,Portuguese,NaN,NaN,...,2.0,10.500000,0.0,0.0,5.0,5.0,0.0,0.000000,0.0,75
3186,single,2.0,2.0,Informatics Engineering,1.0,12th grade,116.0,Portuguese,9th grade,12th grade,...,0.0,0.000000,0.0,0.0,5.0,0.0,0.0,0.000000,0.0,75
2070,single,2.0,6.0,Management,1.0,12th grade,118.0,Portuguese,4th grade,12th grade,...,3.0,10.000000,1.0,0.0,5.0,9.0,3.0,12.333333,0.0,15
3286,divorced,0.0,1.0,Management (evening attendance),0.0,12th grade,140.0,Portuguese,4th grade,4th grade,...,5.0,13.800000,0.0,0.0,5.0,6.0,5.0,13.800000,0.0,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4144,single,0.0,1.0,Oral Hygiene,1.0,12th grade,130.0,Portuguese,9th grade,6th grade,...,5.0,12.400000,0.0,0.0,8.0,14.0,7.0,12.150000,0.0,44
2761,single,0.0,1.0,Nursing,1.0,Master degree,150.0,Portuguese,Bachelor degree,6th grade,...,6.0,13.046667,0.0,2.0,8.0,10.0,0.0,13.306250,0.0,76
4410,married,0.0,1.0,Management (evening attendance),0.0,12th grade,140.0,Portuguese,4th grade,4th grade,...,5.0,14.600000,0.0,0.0,5.0,6.0,5.0,14.333333,0.0,34


In [115]:
# replace values on columns that have lots of different values that can be grouped together to reduce the total number of dummies created after

for col in ["Mother's qualification",  "Father's qualification", "Previous qualification"]:
    X.replace(regex={col: {r"(?i)^no school.*$": '0',
                           r"(?i)^[0-4][a-z]{2} grade.*$": '1', 
                           r"(?i)^[5-9]th grade.*$": '2', 
                           r"(?i)^1[0-2]th grade.*$": '3', 
                           r"(?i)^incomplete bachelor.*$": '4', 
                           r"(?i)^bachelor degree.*$": '5',
                           r"(?i)^post-grad.*$": '6',
                           r"(?i)^master degree.*$": '7',
                           r"(?i)^phd.*$": '8',}}, inplace=True)
    
    testdata.replace(regex={col: {r"(?i)^no school.*$": '0',
                           r"(?i)^[0-4][a-z]{2} grade.*$": '1', 
                           r"(?i)^[5-9]th grade.*$": '2', 
                           r"(?i)^1[0-2]th grade.*$": '3', 
                           r"(?i)^incomplete bachelor.*$": '4', 
                           r"(?i)^bachelor degree.*$": '5',
                           r"(?i)^post-grad.*$": '6',
                           r"(?i)^master degree.*$": '7',
                           r"(?i)^phd.*$": '8',}}, inplace=True)

for col in ["Mother's occupation", "Father's occupation"]:
    X.replace(to_replace={col: ["Superior-level Professional", "Intermediate-level Professional", "Politician/CEO", "Teacher", "Information Technology Specialist"]}, value="Professional Fields", inplace=True)
    X.replace(to_replace={col: ["Skilled construction workers", "Assembly Worker", "Factory worker", "Lab Technocian"]}, value="Technical and Skilled Trades", inplace=True)
    X.replace(to_replace={col: ["Administrative Staff", "Office worker", "Accounting operator"]}, value="White collar Jobs", inplace=True)
    X.replace(to_replace={col: ["Restaurant worker", "Personal care worker", "Seller", "Cleaning worker"]}, value="Service Industry", inplace=True)
    X.replace(to_replace={col: ["Private Security", "Armed Forces"]}, value="Security and Armed Forces", inplace=True)
    X.replace(to_replace={col: ["Unskilled Worker", "Other", "Student", "Artist"]}, value="Recreational or unskilled", inplace=True)
    X.replace(to_replace={col: ["Engineer", "Scientist", "Health professional"]}, value="STEM Jobs", inplace=True)

    testdata.replace(to_replace={col: ["Superior-level Professional", "Intermediate-level Professional", "Politician/CEO", "Teacher", "Information Technology Specialist"]}, value="Professional Fields", inplace=True)
    testdata.replace(to_replace={col: ["Skilled construction workers", "Assembly Worker", "Factory worker", "Lab Technocian"]}, value="Technical and Skilled Trades", inplace=True)
    testdata.replace(to_replace={col: ["Administrative Staff", "Office worker", "Accounting operator"]}, value="White collar Jobs", inplace=True)
    testdata.replace(to_replace={col: ["Restaurant worker", "Personal care worker", "Seller", "Cleaning worker"]}, value="Service Industry", inplace=True)
    testdata.replace(to_replace={col: ["Private Security", "Armed Forces"]}, value="Security and Armed Forces", inplace=True)
    testdata.replace(to_replace={col: ["Unskilled Worker", "Other", "Student", "Artist"]}, value="Recreational or unskilled", inplace=True)
    testdata.replace(to_replace={col: ["Engineer", "Scientist", "Health professional"]}, value="STEM Jobs", inplace=True)

X.replace(to_replace={"Marital status": {"facto union": "married",
                            "legally separated": "divorced",
                            # "widower": "single" reduces performance
                            }}, inplace=True)

testdata.replace(to_replace={"Marital status": {"facto union": "married",
                            "legally separated": "divorced",
                            # "widower": "single" reduces performance
                            }}, inplace=True)

In [116]:
# get dummies, add 0s to columns that are in test but not in train, and vice-versa, reorder the columns so that they are in the same order

X: pd.DataFrame = pd.get_dummies(data=X, prefix_sep="-", dummy_na=True, drop_first=False)
testdata: pd.DataFrame = pd.get_dummies(data=testdata, prefix_sep="-", dummy_na=True, drop_first=False)

X[(coldiff1 := list(set(X.columns) - set(testdata.columns)))] = 0
testdata[(coldiff2 := list(set(testdata.columns) - set(X.columns)))] = 0

testdata = testdata.reindex(columns=X.columns)

In [ ]:
# Run a model to get the features with least importance
model: RandomForestClassifier = RandomForestClassifier(800)
model.fit(X, y)

predictions = model.predict(testdata)

for x, y in zip(list(X.columns), model.feature_importances_):
    if y*100 < 0.5:
        print(f"{x}: {y*100:.2f}")

In [117]:
# Get predictions
model: RandomForestClassifier = RandomForestClassifier(800, random_state=15)
model.fit(X, y)

predictions = model.predict(testdata)

for x, y in zip(list(X.columns), model.feature_importances_):
    if y*100 < 0.5:
        print(f"{x}: {y*100:.2f}")

outputData = pd.DataFrame([testdata.index, predictions]).T;
outputData.columns = ["Userid", "Success"];

outputData.to_csv("./answer.csv", index=False);

Morning shift participation: 0.39
Special needs: 0.13
International: 0.24
Marital status-divorced: 0.13
Marital status-married: 0.27
Marital status-single: 0.31
Marital status-widower: 0.03
Marital status-nan: 0.00
Course-Advertising and Marketing Management: 0.36
Course-Agronomy: 0.20
Course-Animation and Multimedia Design: 0.47
Course-Biofuel Production Technologies: 0.04
Course-Communication Design: 0.29
Course-Equinculture: 0.25
Course-Informatics Engineering: 0.42
Course-Journalism and Communication: 0.35
Course-Management: 0.44
Course-Management (evening attendance): 0.27
Course-Oral Hygiene: 0.13
Course-Social Service (evening attendance): 0.19
Course-Tourism: 0.32
Course-Veterinary Nursing: 0.41
Course-nan: 0.00
Previous qualification-2: 0.18
Previous qualification-3: 0.38
Previous qualification-4: 0.03
Previous qualification-5: 0.19
Previous qualification-6: 0.06
Previous qualification-7: 0.01
Previous qualification-nan: 0.01
Nationality-Angolan: 0.00
Nationality-Brazilian: 0.

In [ ]:
# Split

In [ ]:
import math, sys, numpy as np, pandas as pd
from typing import Literal;
from sklearn import preprocessing;
from sklearn.ensemble import RandomForestClassifier;

def encodeCols(trainData: pd.DataFrame, testData: pd.DataFrame, columns: list[str], /):
  """Encode categorical columns to numerical.

  Args:
      trainData (pd.DataFrame): The train dataframe where conversion is to be performed.
      testData (pd.DataFrame): The test dataframe where conversion is to be performed.
      columns (list[str]): The columns to be converted.
  """;

  for (col) in columns:
    le = preprocessing.LabelEncoder();
    dfCombined = le.fit_transform(pd.concat([trainData[col], testData[col]], axis=0));

    trainLength = trainData[col].__len__();

    # dfCombined.

    trainData[col] = dfCombined[:trainLength];
    testData[col] = dfCombined[trainLength:];
  # end for
# end encodeCols

def importData(trainPath: str = "./data/train.csv", testPath: str = "./data/test.csv") -> tuple[ tuple[pd.DataFrame, pd.Series], pd.DataFrame ]:
  """Imports the training and testing dataset and returns a training feature/target tuple and the testing features.

  Args:
    `trainPath` (`str`): the path (relative or absolute) to the training data csv.
    `testPath` (`str`): the path (relative or absolute) to the testing data csv.

  Returns:
    `tuple[tuple[pd.DataFrame, pd.Series], pd.DataFrame]` - `( (trainX, trainY), testX )`
  """;

  userIdArray: list[Literal["Userid"]] = ["Userid"];

  trainData: pd.DataFrame = pd.read_csv(trainPath, sep=",").drop("Observations", axis=1).drop_duplicates().set_index("Userid");
  # pd.DataFrame.it.drop_duplicates() only removes rows where all values are the same; does not drop if different values but same userId.

  testX: pd.DataFrame = pd.read_csv(testPath, sep=",").drop("Observations", axis=1).set_index(userIdArray);
  if (testX.duplicates().__len__() > 0): raise ValueError("Test data contains duplicate user IDss.");

  trainY: pd.Series = trainData["Success"];
  trainX: pd.Series = trainData.drop(["Success"], axis = 1);

  del trainData, userIdArray;

  return (
    (trainX, trainY),
    testX
  );
#end importData

def main() -> int:
  userIdArray: list[Literal["Userid"]] = ["Userid"];
  trainData: pd.DataFrame = pd.read_csv(path + "project_data/train.csv", sep=",").drop("Observations", axis=1).set_index(userIdArray);
  testData: pd.DataFrame = pd.read_csv(path + "project_data/test.csv", sep=",").drop("Observations", axis=1).set_index(userIdArray);

  del userIdArray;

  # trainData.index = testData.index = ["Userid"];

  # trainData.groupby("Userid");

  # rows without a "registered" value in the dataset are garbage (userId & outcome only, no use for ML algo).
  trainData.drop(trainData[trainData["Registered"] != "Yes"].index, inplace=True);

  trainData = trainData.drop("Registered", axis=1);
  testData = testData.drop("Registered", axis=1);


  encodeCols(trainData, testData, [
    "Marital status",
    "Course",
    "Previous qualification",
    "Nationality",
    "Mother's qualification",
    "Father's qualification",
    "Mother's occupation",
    "Father's occupation"
  ]);

  # drop = [
  #   "Nationality",
  #   "International",
  #   "Special needs",
  #   "Displaced",
  #   "Morning shift participation",
  #   "Previous qualification",
  #   "Marital status",
  #   'Application mode',
  #   'Application order',
  #   'Course',
  #   'Previous qualification score',
  #   "Mother's qualification",
  #   "Father's qualification",
  #   "Mother's occupation",
  #   "Father's occupation",
  #   #'Entry score',
  #   'Debtor',
  #   'Regularized Fees',
  #   'Gender_Male',
  #   'External Funding',
  #   'Age at enrollment',
  #   'N units credited 1st period',
  #   'N units taken 1st period',
  #   'N scored units 1st period',
  #   #'N units approved 1st period',
  #   #'Average grade 1st period',
  #   'N unscored units 1st period',
  #   'N units credited 2nd period',
  #   'N units taken 2nd period',
  #   'N scored units 2nd period',
  #   #'N units approved 2nd period',
  #   #'Average grade 2nd period',
  #   'N unscored units 2nd period',
  #   #'Social Popularity'
  # ];

  # trainData = trainData.drop(drop, axis=1);
  # testData = testData.drop(drop, axis=1);


  trainX: pd.DataFrame = trainData.drop("Success", axis=1);
  trainY: pd.Series = trainData["Success"];

  model: RandomForestClassifier = RandomForestClassifier(800);
  model.fit(trainX, trainY);

  predictions = model.predict(testData);

  for x, y in zip(list(trainX.columns), model.feature_importances_):
    print(f"{x}: {y*100:.2f}");

  outputData = pd.DataFrame([testData.index, predictions]).T;
  outputData.columns = ["Userid", "Success"];

  outputData.to_csv("./answer.csv", index=False);

  return 0;
# end main
main()